# 02 Make Walk Network

In [1]:
import os
import sys
import yaml
from subprocess import Popen
import pickle

import pandas as pd
import geopandas as gpd
import numpy as np


from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import WranglerLogger
from network_wrangler import ProjectCard

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

### Remote I/O and Parameters

In [4]:
lasso_dir = os.path.join('C:/Users', 'david.ory', 'Documents', 'GitHub', 'lasso')
input_dir = os.path.join(lasso_dir, 'Examples', 'stpaul')
pickle_dir = input_dir
output_dir = input_dir

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-03-12 08:45:09, INFO: Lasso base directory set as: C:/Users\david.ory\Documents\GitHub\lasso
2021-03-12 08:45:09, INFO: Lasso base directory set as: C:/Users\david.ory\Documents\GitHub\lasso


### Read Version 00 Scenario

In [6]:
version_00_pickle_file_name = os.path.join(pickle_dir, 'working_scenario_full_00.pickle')
v_01_scenario = pickle.load(open(version_00_pickle_file_name, 'rb'))

### Make Travel Model Networks

In [7]:
m_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v_01_scenario.road_net, 
    parameters = parameters)

2021-03-12 08:45:41, INFO: Lasso base directory set as: C:/Users\david.ory\Documents\GitHub\lasso
2021-03-12 08:45:41, INFO: Lasso base directory set as: C:/Users\david.ory\Documents\GitHub\lasso


In [8]:
m_net.roadway_standard_to_met_council_network()

2021-03-12 08:45:41, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2021-03-12 08:45:41, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2021-03-12 08:45:41, INFO: Didn't detect managed lanes in network.
2021-03-12 08:45:41, INFO: Didn't detect managed lanes in network.
2021-03-12 08:45:41, INFO: Creating calculated roadway variables.
2021-03-12 08:45:41, INFO: Creating calculated roadway variables.
2021-03-12 08:45:41, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type
2021-03-12 08:45:41, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type


c:\users\david.ory\documents\github\lasso\lasso\roadway.py:423: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2021-03-12 08:46:04, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2021-03-12 08:46:04, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2021-03-12 08:46:04, INFO: Adding roadway network variable for county using a spatial join with: C:/Users\david.ory\Documents\GitHub\lasso\metcouncil_data\county\cb_2017_us_county_5m.shp
2021-03-12 08:46:04, INFO: Adding roadway network variable for county using a spatial join with: C:/Users\david.ory\Documents\GitHub\lasso\metcouncil_data\county\cb_2017_us_county_5m.shp


c:\users\david.ory\documents\github\lasso\lasso\roadway.py:288: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2021-03-12 08:46:18, INFO: Finished Calculating county variable: county
2021-03-12 08:46:18, INFO: Finished Calculating county variable: county
2021-03-12 08:46:19, INFO: Calculating MPO as roadway network variable: mpo
2021-03-12 08:46:19, INFO: Calculating MPO as roadway network variable: mpo
2021-03-12 08:46:19, INFO: Finished calculating MPO variable: mpo
2021-03-12 08:46:19, INFO: Finished calculating MPO variable: mpo
2021-03-12 08:46:19, INFO: Adding Counts
2021-03-12 08:46:19, INFO: Adding Counts
2021-03-12 08:46:19, INFO: Adding Variable AADT using Shared Streets Reference from C:/Users\david.ory\Documents\GitHub\lasso\metcouncil_data\count_mn\mn_count_ShSt_API_match.csv
2021-03-12 08:46:19, INFO: Adding Variable AADT using Shared Streets Reference from C:/Users\david.ory\Documents\GitHub\lasso\metcouncil_data\count_mn\mn_count_ShSt_API_match.csv
2021-03-12 08:46:19, INFO: Added variable: AADT using Shared Streets Reference
2021-03-12 08:46:19, INFO: Added variable: AADT using

ValueError: could not convert string to float: 

## Write to Disk

### As Shapefile

In [ ]:
out_cols = ['model_link_id', 'shape_id', 'assign_group', 'drive_access', 'roadway_class',
            'lanes_AM', 'lanes_MD', 'lanes_PM', 'lanes_NT',
            'roadway', 'ramp_flag', 'geometry']

m_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'v01_links.shp'),
    output_node_shp = os.path.join(output_dir, 'v01_nodes.shp'),
    link_output_variables = out_cols,
    data_to_csv = False,
    data_to_dbf = True,
)

### Write Fixed Width (for Cube)

In [ ]:
m_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'v01_links.txt'),
    output_node_txt = os.path.join(output_dir, 'v01_nodes.txt'),
    link_output_variables = out_cols,
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
)

In [ ]:
standard_transit_net = StandardTransit.fromTransitNetwork(v_01_scenario.transit_net, parameters = parameters)
standard_transit_net.write_as_cube_lin(outpath = os.path.join(output_dir, "v01_transit.lin"))